# Treinamento de Modelo: [NOME DO ALGORITMO]
> **Fase 3 - Time 7**

**Responsável:** [Seu Nome]
**Algoritmo:** [Ex: Random Forest, SVM, KNN...]

**Objetivos deste Notebook:**
1.  **Carregar:** Importar os dados de treino/teste padronizados (do pré-processamento unificado).
2.  **Treinar:** Ajustar o modelo [NOME DO ALGORITMO] aos dados de treino.
3.  **Avaliar:** Medir a performance com foco no **Recall** (Sensibilidade) para detectar doenças.
4.  **Validar:** Testar se o modelo consegue identificar o grupo de risco "Assintomático".

**Métricas Chave:**
* **Recall (Sensibilidade):** Prioridade máxima. Não podemos deixar doentes irem para casa.
* **F1-Score:** Para garantir que não estamos apenas "chutando que todos estão doentes".

## 1. Configuração de ambiente
Importação de bibliotecas e definição de estilos gráficos padronizados.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import os

# Métricas padronizadas
from sklearn.metrics import classification_report, confusion_matrix, recall_score, roc_auc_score, roc_curve, ConfusionMatrixDisplay

# Configuração visual
sns.set_theme(style="whitegrid")
palette = sns.color_palette("coolwarm", as_cmap=True)

In [2]:
# Carregamento dos Dados Processados (Pasta data/processed)
# IMPORTANTE: Não mexer neste bloco para garantir compatibilidade!

try:
    X_train = joblib.load('../data/processed/X_train.pkl')
    y_train = joblib.load('../data/processed/y_train.pkl')
    X_test = joblib.load('../data/processed/X_test.pkl')
    y_test = joblib.load('../data/processed/y_test.pkl')
    feature_names = joblib.load('../data/processed/feature_names.pkl')
    
    print(f"✅ Dados carregados com sucesso!")
    print(f"Treino: {X_train.shape}")
    print(f"Teste:  {X_test.shape}")
except FileNotFoundError:
    print("❌ Arquivos não encontrados. Rode o notebook '02_preprocessing_unificado.ipynb' primeiro.")

✅ Dados carregados com sucesso!
Treino: (237, 22)
Teste:  (60, 22)


## 2. Treinamento do Modelo

Adotaremos uma abordagem em duas etapas:
1.  **Baseline:** Treinar o modelo com parâmetros padrão (default) para estabelecer uma linha de base.
2.  **Otimização:** Encontrar a melhor combinação de hiperparâmetros focada em **Recall**.

### 2.1 Baseline

In [ ]:
# ==========================================================
# 2.1 BASELINE (Modelo "Cru")
# ==========================================================

# 1. Importe seu modelo aqui
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier

# 2. Instancie com parâmetros padrão (apenas random_state se aplicável)
# model_baseline = SVC(random_state=42)
model_baseline = [SEU_MODELO_AQUI](random_state=42)

print("🚀 Treinando Baseline...")
model_baseline.fit(X_train, y_train)

# Avaliação Rápida do Baseline
y_pred_base = model_baseline.predict(X_test)
recall_base = recall_score(y_test, y_pred_base)
print(f"✅ Baseline treinado! Recall inicial: {recall_base:.2%}")

### 2.2 Otimização

⚡ Escolha sua Estratégia de Otimização

Você pode escolher entre **Grid Search** (tenta todas as combinações, mais lento, mais preciso) ou **Random Search** (tenta combinações aleatórias, mais rápido, bom para muitos parâmetros).

> **Dica:**
> * Use **GridSearch** se tiver poucos parâmetros (ex: KNN, Regressão Logística).
> * Use **RandomSearch** se tiver muitos parâmetros ou se o Grid estiver demorando muito (ex: Random Forest, SVM).

In [ ]:
# ==========================================================
# 2.2 OTIMIZAÇÃO DE HIPERPARÂMETROS
# EXEMPLO DE USO COM GRIDSEARCHCV
# ==========================================================
from sklearn.model_selection import GridSearchCV

# A. Defina o Grid de Parâmetros (Consulte a documentação do seu modelo!)
# Exemplo para SVM: {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
# Exemplo para Random Forest: {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, None]}
# Exemplo para KNN: {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}

param_grid = {
    # [PREENCHA AQUI COM OS PARÂMETROS DO SEU ALGORITMO]
}

# B. Configuração da Busca
# scoring='recall': Dizemos ao GridSearch que o mais importante é ACERTAR OS DOENTES
grid_search = GridSearchCV(
    estimator=[SEU_MODELO_AQUI](random_state=42),
    param_grid=param_grid,
    cv=5,               # Validação Cruzada em 5 partes
    scoring='recall',   # Foco total em Recall
    n_jobs=-1,          # Usa todos os processadores
    verbose=1
)

print("🕵️ Iniciando busca pelos melhores parâmetros (pode demorar)...")
grid_search.fit(X_train, y_train)

# C. Melhores Resultados
best_model = grid_search.best_estimator_

print("-" * 30)
print(f"🏆 Melhor Recall na Validação: {grid_search.best_score_:.2%}")
print(f"⚙️ Melhores Parâmetros: {grid_search.best_params_}")

In [ ]:
# ==========================================================
# 2.2 (OPÇÃO B) RANDOMIZED SEARCH CV
# Use esta célula se o GridSearch estiver demorando muito!
# ==========================================================
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# A. Defina a Distribuição de Parâmetros
# Diferente do Grid, aqui podemos passar listas ou distribuições estatísticas
param_dist = {
    # [COLE SEUS PARÂMETROS AQUI - Igual ao Grid]
    # Exemplo para Random Forest:
    # 'n_estimators': randint(50, 300),
    # 'max_depth': [5, 10, 20, None],
}

if param_dist: # Só roda se você preencheu o dicionário
    random_search = RandomizedSearchCV(
        estimator=[SEU_MODELO_AQUI](random_state=42),
        param_distributions=param_dist,
        n_iter=50,          # Tenta 50 combinações aleatórias (Ajuste conforme sua paciência)
        cv=5,               # Validação Cruzada
        scoring='recall',   # Foco no Recall
        n_jobs=-1,
        random_state=42,    # Garante que o aleatório seja sempre o mesmo
        verbose=1
    )

    print("🎲 Iniciando busca aleatória (Random Search)...")
    random_search.fit(X_train, y_train)

    # Atualiza o best_model com o vencedor do Random Search
    best_model = random_search.best_estimator_

    print("-" * 30)
    print(f"🏆 Melhor Recall (Random): {random_search.best_score_:.2%}")
    print(f"⚙️ Melhores Parâmetros: {random_search.best_params_}")
else:
    print("⚠️ Defina o param_dist para rodar o Random Search.")

## 3. Avaliação de Performance

In [ ]:
# Previsões do Melhor Modelo
y_pred_final = best_model.predict(X_test)
recall_final = recall_score(y_test, y_pred_final)

print(f"📈 Evolução do Recall:")
print(f"   - Baseline:  {recall_base:.2%}")
print(f"   - Otimizado: {recall_final:.2%}")
print(f"   - Ganho:     {(recall_final - recall_base):.2%}")

# Matriz de Confusão do Modelo Campeão
plt.figure(figsize=(6, 5))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_final, cmap='Blues', display_labels=['Saudável', 'Doente'])
plt.title('Matriz de Confusão (Melhor Modelo)')
plt.grid(False)
plt.show()

# Classification Report Completo
print(classification_report(y_test, y_pred_final))

### 3.1. Avalição de Grupo Assintomático
Nesta etapa, validaremos a tese central: Verificaremos a performance do modelo especificamente nos pacientes que **não apresentavam dor típica** (CP=3: Asymptomatic).

> **Por que isso é importante?**
> A maioria dos erros humanos acontece aqui. Se nosso modelo tiver um bom Recall neste grupo, provamos que a IA enxerga além do óbvio.

In [ ]:
# ==========================================================
# 3.1 VALIDAÇÃO NO SUBGRUPO ASSINTOMÁTICO
# ==========================================================

# 1. Identificando a coluna correta (pós One-Hot Encoding)
# O nome da coluna geralmente é 'cp_3.0' ou 'cp_3', dependendo do processamento
cols_assintomaticas = [col for col in X_test.columns if 'cp_3' in col]

if len(cols_assintomaticas) > 0:
    col_alvo = cols_assintomaticas[0]
    print(f"🔎 Analisando subgrupo oculto: '{col_alvo}'")

    # 2. Filtrando os dados
    # Pegamos apenas as linhas onde o paciente é assintomático (Valor 1 na coluna)
    mask_assin = X_test[col_alvo] == 1
    
    X_test_sub = X_test[mask_assin]
    y_test_sub = y_test[mask_assin]
    
    if len(X_test_sub) > 0:
        # 3. Avaliando o Melhor Modelo neste subgrupo
        y_pred_sub = best_model.predict(X_test_sub)
        
        recall_sub = recall_score(y_test_sub, y_pred_sub)
        acc_sub = sum(y_test_sub == y_pred_sub) / len(y_test_sub)
        
        print("-" * 40)
        print(f"📊 RESULTADO DO TESTE DE FOGO:")
        print(f"   Pacientes Assintomáticos no Teste: {len(X_test_sub)}")
        print(f"   Recall (Doentes detectados): {recall_sub:.2%} 🎯")
        print("-" * 40)
        
        if recall_sub > 0.80:
            print("✅ SUCESSO: O modelo é excelente em detectar isquemia silenciosa!")
        else:
            print("⚠️ ATENÇÃO: O modelo tem dificuldade com casos sem dor. Considere ajustar o threshold.")
    else:
        print("ℹ️ Nenhum paciente assintomático caiu na amostragem de teste atual.")
else:
    print("❌ Coluna de 'cp_3' não encontrada. Verifique os nomes das features.")

## 4. Exportação do Modelo
Salvando o modelo campeão (otimizado) para uso na aplicação final.

In [ ]:
# Cria o diretório se não existir
os.makedirs('../models', exist_ok=True)

# Define o nome do arquivo baseado no algoritmo usado
# Ex: modelo_RandomForestClassifier.pkl
nome_modelo = best_model.__class__.__name__
caminho_final = f'../models/modelo_{nome_modelo}.pkl'

joblib.dump(best_model, caminho_final)

print(f"💾 Modelo salvo com sucesso!")
print(f"📂 Local: {caminho_final}")